## ✫FineTuning済みのBERTモデルを使用して文章のベクトル化

### 1. 文章の本文をベクトル化するために学習済みのBERTモデルをロード

In [1]:
import sentence_transformers
from sentence_transformers import SentenceTransformer

In [2]:
sbert = SentenceTransformer('./sbert')
vector = sbert.encode('100回分のループを渡しているので, 1回のループで1%の進捗があります.')

In [3]:
vector.shape
# (768,)

(768,)

### 2. 文章の本文をリスト化

In [4]:
import os
# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

In [5]:
def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

In [6]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

### 3. Livedoorテキストの各本文をベクトル化して、以下の3つを作成する
- 文章ベクトル → 文章本文の辞書
- （文章ID, 文章ベクトル）を配列としたリスト
- 文章本文 → 文章ラベルの辞書

In [7]:
import numpy as np
livedoor_vec_to_text_dict = {}
livedoor_vec_list = []
livedoor_text_to_label_dict = {}

In [8]:
from tqdm import tqdm 

for text_item, label_item in tqdm(zip(list_text, list_label)):
    vector = sbert.encode(text_item)
    n = np.array(vector)
    n = tuple(n)
    livedoor_vec_to_text_dict[n] = text_item
    livedoor_vec_list.append((len(livedoor_vec_list), n))
    livedoor_text_to_label_dict[text_item] = label_item

7376it [04:00, 30.64it/s]


In [9]:
print(len(list_text))
print(len(livedoor_vec_to_text_dict))
print(len(livedoor_vec_list))
print(len(livedoor_text_to_label_dict))
# 7376
# 7365 → 同じものがあるのか？
# 7376
# 7368 → 同じものがあるのか？

7376
7365
7376
7368


In [10]:
# 作成した辞書（文章ベクトル→文章本文）の検証
vector = sbert.encode(list_text[10])
n = np.array(vector)
n = tuple(n)
livedoor_vec_to_text_dict[n]

'独女のココロを優しく癒すゆるキャラたち。ドアラ、ひこにゃん、せんとくんなどなど、そのクオリティの如何に関わらず、なぜか「大ブレイク」してしまうところもゆるキャラの魅力。一度火がつけばその人気は一気に全国区となり、関連商品は飛ぶように売れまくる。そのゆるキャラ界で新たなスターとなりそうな1匹（？）こそ、ダイハツ自動車「ムーヴコンテ」のCMに登場している「カクカクシカジカ」だろう。コミカルな声にあわせ「カクカクシカジカ」と軽妙な動きを見せるシカ——、といえば、一度は目にしたこともあるはず。初登場は、新車ムーヴコンテが発表された08年8月のCMで、「このカクカクしたクルマ、いいところがシカジカ。ゆえにカクカクシカジカ」と、クルマの説明なのか、自己紹介なのかすらも不明なセリフで登場。以降、自分がシカであるくせに、干支の牛や虎になるために気ぐるみを着てみたり、ムーヴコンテに搭載されている新技術の「CVT」を「ちょっと ヴぃっくりな 低燃費」と解説するなど、ちょっとひねくれた辛口評論家という設定を無視するかのごとく、キャラが独走を続けている。そのゆるキャラのネクストジェネレーションであるカクカクシカジカのオリジナルグッズがもらえるキャンペーン「カクカクトクトクフェア」が、6月5日（土）、6日（日）の両日で開催される。期間中にダイハツのお店に来店すると、抽選で「カキ氷器」「折りたたみ傘」「ペットッボトルホルダー」がもれなく当たるという。フェア告知用のCMも製作されているようで、今回、カクカクシカジカが扮するのはサッカーの審判。ワールドカップというタイミングに合わせての演出のようだが、今作も適当なゆるさを楽しませてくれるはず。CMは本日よりOA。「かわいらしく、スタイリッシュな軽自動車」というブランドイメージを持つダイハツが、さらに女性からの支持率を上がりそうなゆるキャラの躍進。この週末には普段はなかなか行く機会がないディーラーにゆるキャラ目当てで行ってみるのもいいかもしれない。・カクカクトクトクフェア・奔放な振る舞いを見られる過去のCMアーカイブ【コチラ】■「カクカクトクトクフェア」2010年6月5日（土）〜6月6日（日）全国のダイハツのお店※一部のお店では実施していない場合がございます。■景品1等 カクカクシカジカカキ氷器2等 カサカサシカジカ折りたたみ傘3等 ゴクゴクシカジ

In [11]:
# 10番目の文章とラベルを確認
print(list_text[10])
print(list_label[10])

独女のココロを優しく癒すゆるキャラたち。ドアラ、ひこにゃん、せんとくんなどなど、そのクオリティの如何に関わらず、なぜか「大ブレイク」してしまうところもゆるキャラの魅力。一度火がつけばその人気は一気に全国区となり、関連商品は飛ぶように売れまくる。そのゆるキャラ界で新たなスターとなりそうな1匹（？）こそ、ダイハツ自動車「ムーヴコンテ」のCMに登場している「カクカクシカジカ」だろう。コミカルな声にあわせ「カクカクシカジカ」と軽妙な動きを見せるシカ——、といえば、一度は目にしたこともあるはず。初登場は、新車ムーヴコンテが発表された08年8月のCMで、「このカクカクしたクルマ、いいところがシカジカ。ゆえにカクカクシカジカ」と、クルマの説明なのか、自己紹介なのかすらも不明なセリフで登場。以降、自分がシカであるくせに、干支の牛や虎になるために気ぐるみを着てみたり、ムーヴコンテに搭載されている新技術の「CVT」を「ちょっと ヴぃっくりな 低燃費」と解説するなど、ちょっとひねくれた辛口評論家という設定を無視するかのごとく、キャラが独走を続けている。そのゆるキャラのネクストジェネレーションであるカクカクシカジカのオリジナルグッズがもらえるキャンペーン「カクカクトクトクフェア」が、6月5日（土）、6日（日）の両日で開催される。期間中にダイハツのお店に来店すると、抽選で「カキ氷器」「折りたたみ傘」「ペットッボトルホルダー」がもれなく当たるという。フェア告知用のCMも製作されているようで、今回、カクカクシカジカが扮するのはサッカーの審判。ワールドカップというタイミングに合わせての演出のようだが、今作も適当なゆるさを楽しませてくれるはず。CMは本日よりOA。「かわいらしく、スタイリッシュな軽自動車」というブランドイメージを持つダイハツが、さらに女性からの支持率を上がりそうなゆるキャラの躍進。この週末には普段はなかなか行く機会がないディーラーにゆるキャラ目当てで行ってみるのもいいかもしれない。・カクカクトクトクフェア・奔放な振る舞いを見られる過去のCMアーカイブ【コチラ】■「カクカクトクトクフェア」2010年6月5日（土）〜6月6日（日）全国のダイハツのお店※一部のお店では実施していない場合がございます。■景品1等 カクカクシカジカカキ氷器2等 カサカサシカジカ折りたたみ傘3等 ゴクゴクシカジカ

In [12]:
# 作成した辞書（文章ベクトル→文章本文）の検証
vector = sbert.encode(list_text[1015])
n = np.array(vector)
n = tuple(n)
livedoor_vec_to_text_dict[n]

'スペインで開催されていた世界最大のモバイル・通信関連イベント「Mobile World Congress 2012(MWC2012)」が閉幕した。会期中は各メーカーが連日新製品発表会を開催していたが、圧倒的な製品数で話題となったのが中国のZTEだ。発表会で紹介されたスマートフォン新製品は8機種。それに加えてタブレットや開発中の製品なども紹介され、同社のブースにはそれらの製品がずらりと展示されていた。4Gからハイエンドまでスマートフォン新製品で埋め尽くされたMWC2012のZTEブースZTEの新製品発表会で、同社端末部門トップの何士友執行副総裁は「2015年までに世界シェア3位になる」と同社の目標を明言。同社の上に位置するAppleとの差はまだ開いているが、1製品だけで勝負するAppleに対し、エントリーフィーチャーフォンから多機能スマートフォンまで手がけ、しかも成長を続ける新興市場にも強いZTEが販売数を抜くチャンスは十分にある。なお、調査会社ガートナーによれば、2011年通期の端末販売台数はAppleが8926万台、LGが約8637万台、ZTEは5688万台であった。MWC2012で発表したZTEの新製品は、そのいずれもが明確なターゲットとポジションを持った製品だ。2015年のシェア3位入りを前に、まず2012-13年にシェア4位を目標とする戦略的な製品群を揃えた、と言えるだろう。同社の新製品発表会は大手メーカーのような派手な演出は行われなかったが、次々と発表される製品ラッシュには会場を埋め尽くした世界中のメディア関係者からも驚嘆の声が飛んでいた。クアッドコアやLTE対応など、ZTEの新製品は多岐に及ぶでは、ZTEの新製品はそれぞれどのような戦略を持った製品なのか。まずは同社が弱いハイエンド製品が3機種も登場する。「ZTE Era」はNVIDIAのTegra3を搭載しながらも厚みは7.8mmと薄い。また「PF200」と「PF910」はLTE対応スマートフォンで、複数の周波数にも対応。そして「PF112 HD」は4.5インチ1280x720ピクセルのフルHDディスプレイを搭載する。いずれも他社の同等スペック品より15%程度安い価格になる見込みだと言う。このスペックと価格は通信事業者が自社ブランドのハイエンドスマートフォンとしてラインナップに加えたくなる魅力も持って

In [13]:
# 15番目の文章とラベルを確認
print(list_text[1015])
print(list_label[1015])

スペインで開催されていた世界最大のモバイル・通信関連イベント「Mobile World Congress 2012(MWC2012)」が閉幕した。会期中は各メーカーが連日新製品発表会を開催していたが、圧倒的な製品数で話題となったのが中国のZTEだ。発表会で紹介されたスマートフォン新製品は8機種。それに加えてタブレットや開発中の製品なども紹介され、同社のブースにはそれらの製品がずらりと展示されていた。4Gからハイエンドまでスマートフォン新製品で埋め尽くされたMWC2012のZTEブースZTEの新製品発表会で、同社端末部門トップの何士友執行副総裁は「2015年までに世界シェア3位になる」と同社の目標を明言。同社の上に位置するAppleとの差はまだ開いているが、1製品だけで勝負するAppleに対し、エントリーフィーチャーフォンから多機能スマートフォンまで手がけ、しかも成長を続ける新興市場にも強いZTEが販売数を抜くチャンスは十分にある。なお、調査会社ガートナーによれば、2011年通期の端末販売台数はAppleが8926万台、LGが約8637万台、ZTEは5688万台であった。MWC2012で発表したZTEの新製品は、そのいずれもが明確なターゲットとポジションを持った製品だ。2015年のシェア3位入りを前に、まず2012-13年にシェア4位を目標とする戦略的な製品群を揃えた、と言えるだろう。同社の新製品発表会は大手メーカーのような派手な演出は行われなかったが、次々と発表される製品ラッシュには会場を埋め尽くした世界中のメディア関係者からも驚嘆の声が飛んでいた。クアッドコアやLTE対応など、ZTEの新製品は多岐に及ぶでは、ZTEの新製品はそれぞれどのような戦略を持った製品なのか。まずは同社が弱いハイエンド製品が3機種も登場する。「ZTE Era」はNVIDIAのTegra3を搭載しながらも厚みは7.8mmと薄い。また「PF200」と「PF910」はLTE対応スマートフォンで、複数の周波数にも対応。そして「PF112 HD」は4.5インチ1280x720ピクセルのフルHDディスプレイを搭載する。いずれも他社の同等スペック品より15%程度安い価格になる見込みだと言う。このスペックと価格は通信事業者が自社ブランドのハイエンドスマートフォンとしてラインナップに加えたくなる魅力も持ってい

### 4. 作成した辞書の永続化

In [14]:
import pickle
with open("livedoor_vec_to_text_dict.pkl","wb") as f:
    pickle.dump(livedoor_vec_to_text_dict, f)

with open("livedoor_text_to_label_dict.pkl","wb") as f:
    pickle.dump(livedoor_text_to_label_dict, f)   

### 5. ベクトルのリストの永続化

In [15]:
import pickle
with open("livedoor_vec_list.pkl","wb") as f:
    pickle.dump(livedoor_vec_list, f)